In [1]:
cd /content/drive/My Drive/Spam Classifier

/content/drive/My Drive/Spam Classifier


In [0]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn.metrics as m



In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
dataset=pd.read_csv('spam.csv',encoding='latin-1')
dataset

v1  ... Unnamed: 4
0      ham  ...        NaN
1      ham  ...        NaN
2     spam  ...        NaN
3      ham  ...        NaN
4      ham  ...        NaN
...    ...  ...        ...
5567  spam  ...        NaN
5568   ham  ...        NaN
5569   ham  ...        NaN
5570   ham  ...        NaN
5571   ham  ...        NaN

[5572 rows x 5 columns]

In [0]:
sent=dataset.iloc[:,[1]]['v2']

In [8]:
sent

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [0]:
labels=dataset.iloc[:,[0]]['v1']

In [10]:
labels

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le=LabelEncoder()
labels=le.fit_transform(labels)

In [13]:
le.classes_

array(['ham', 'spam'], dtype=object)

In [14]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [0]:
from keras.utils.np_utils import to_categorical
labels=to_categorical(labels)

In [86]:
labels

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [0]:
import re

In [0]:
lemma=WordNetLemmatizer()

In [0]:
sentences=[]
for j in range(0,len(sent)):
  s=re.sub('[^A-Za-z]',' ',sent[j])
  s=s.lower()
  words=word_tokenize(s)
  words=[lemma.lemmatize(i) for i in words if i not in stopwords.words('english')]
  s=' '.join(words)
  sentences.append(s)

In [90]:
sentences

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune',
 'winner valued network customer selected receivea prize reward claim call claim code kl valid hour',
 'mobile month u r entitled update latest colour mobile camera free call mobile update co free',
 'gon na home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash pound txt csh send cost p day day tsandcs apply reply hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw

In [0]:
from keras.preprocessing.text import one_hot

In [0]:
one_hot_sent=[one_hot(i,10000) for i in sentences]

In [0]:
maxi=[len(i) for i in one_hot_sent]

In [94]:
max(maxi)

77

In [0]:
from keras.preprocessing.sequence import  pad_sequences

In [0]:
padsequences=pad_sequences(one_hot_sent,maxlen=80)

In [97]:
padsequences

array([[   0,    0,    0, ..., 5198, 9708, 3606],
       [   0,    0,    0, ..., 6878,  257, 7238],
       [   0,    0,    0, ..., 8162, 3840, 7417],
       ...,
       [   0,    0,    0, ..., 7844, 3089, 7692],
       [   0,    0,    0, ..., 4967,  257, 5085],
       [   0,    0,    0, ..., 5024, 6280, 6576]], dtype=int32)

In [0]:
feature_train,feature_test,label_train,label_test=train_test_split(padsequences,labels,test_size=0.2,random_state=7)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout,Flatten

In [0]:
model=Sequential()
model.add(Embedding(10000,64,input_length=80))
model.add(LSTM(100))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [101]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 80, 64)            640000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total params: 706,202
Trainable params: 706,202
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(feature_train,label_train,epochs=20,validation_data=(feature_test,label_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4457 samples, validate on 1115 samples
Epoch 1/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.1977 - accuracy: 0.9354 - val_loss: 0.0523 - val_accuracy: 0.9857
Epoch 2/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0353 - accuracy: 0.9906 - val_loss: 0.0465 - val_accuracy: 0.9874
Epoch 3/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0179 - accuracy: 0.9950 - val_loss: 0.0470 - val_accuracy: 0.9857
Epoch 4/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0080 - accuracy: 0.9979 - val_loss: 0.0496 - val_accuracy: 0.9852
Epoch 5/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0048 - accuracy: 0.9991 - val_loss: 0.0579 - val_accuracy: 0.9794
Epoch 6/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.0565 - val_accuracy: 0.9798
Epoch 7/20
4457/4457 [==============================] - 19s 4ms/step - loss: 0.002

In [0]:
label_pred=model.predict(feature_test)

In [104]:
label_pred

array([[9.9999821e-01, 1.6365198e-06],
       [9.9592268e-01, 3.2489148e-03],
       [9.9999893e-01, 1.0859128e-06],
       ...,
       [9.9999487e-01, 4.8116722e-06],
       [9.9999893e-01, 1.0533463e-06],
       [9.9999821e-01, 1.7002430e-06]], dtype=float32)

In [0]:
label_pred_=[np.argmax(i,axis=0) for i in label_pred]

In [108]:
label_pred_[0:5]

[0, 0, 0, 0, 0]

In [0]:
label_test_=[np.argmax(i,axis=0) for i in label_test]

In [111]:
label_test_[0:5]

[0, 0, 0, 0, 0]

In [112]:
m.accuracy_score(label_test_,label_pred_)

0.97847533632287

In [113]:
print(m.classification_report(label_test_,label_pred_))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       970
           1       0.92      0.92      0.92       145

    accuracy                           0.98      1115
   macro avg       0.95      0.95      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [114]:
print(m.confusion_matrix(label_test_,label_pred_))

[[958  12]
 [ 12 133]]
